In [1]:
!pip install langchain-google-genai langchain-core langgraph pydantic google-generativeai --quiet
!pip install -q -U google-genai python-dotenv google-generativeai

In [39]:
# Setting up LLM
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

def llm_call(prompt: str) -> str:
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite", 
        contents=prompt,
        config=types.GenerateContentConfig(
            # It forces the model to output raw JSON only
            response_mime_type="application/json" 
        )
    )
    return response.text

# if __name__ == "__main__":
#     print(llm_call("Hello, Gemini!"))

In [36]:
# Setting up Gmail
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# 1. SETUP GMAIL AUTHENTICATION
SCOPES = [
    'https://www.googleapis.com/auth/gmail.modify',
    'https://www.googleapis.com/auth/calendar'
]

def get_creds():
    creds = None
    if os.path.exists('contents/token.json'):
        creds = Credentials.from_authorized_user_file('contents/token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('contents/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0, open_browser=False) # Modified to not open browser
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

def get_gmail_service():
    creds = get_creds()
    return build('gmail', 'v1', credentials=creds)

def get_calendar_service():
    creds = get_creds()
    return build('calendar', 'v3', credentials=creds)

calendar_service = get_calendar_service()
service = get_gmail_service()

# Check if gmail is connected
profile = service.users().getProfile(userId='me').execute()
print("Email:", profile["emailAddress"])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=344107929802-gujbfh9cebcmdvfmg53v6im3v1iek4rc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50051%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=SJHedqOtmYBeOVgWS1UTxvAOr2khHD&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=344107929802-gujbfh9cebcmdvfmg53v6im3v1iek4rc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50064%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=q2TtNTHGDtrBvy1gcdwhNu71VtIE7e&access_type=offline
Email: forsignins043@gmail.com


In [4]:
def fetch_recent_emails(service, max_results=5):
    results = service.users().messages().list(
        userId="me",
        maxResults=max_results
    ).execute()

    messages = results.get("messages", [])
    full_email_objects = []

    for msg in messages:
        message = service.users().messages().get(
            userId="me",
            id=msg["id"],
            format="full"
        ).execute()
        
        full_email_objects.append(message) 

    return full_email_objects

In [5]:
import base64

def extract_body(payload: dict) -> str:
    body = ""

    if "data" in payload.get("body", {}):
        body = payload["body"]["data"]

    elif "parts" in payload:
        for part in payload["parts"]:
            if part.get("mimeType") == "text/plain" and "data" in part.get("body", {}):
                body = part["body"]["data"]
                break

    if body:
        return base64.urlsafe_b64decode(body).decode("utf-8", errors="ignore")

    return ""


def extract_email_parts(message: dict):
    payload = message.get("payload", {})
    headers = payload.get("headers", [])

    subject = ""
    sender = ""

    for h in headers:
        name = h.get("name", "").lower()
        if name == "subject":
            subject = h.get("value", "")
        elif name == "from":
            sender = h.get("value", "")

    body = extract_body(payload)

    return sender, subject, body


In [6]:
def email_for_llm(sender: str, subject: str, body: str) -> str:
    email_text = f"""
Sender: {sender}
Subject: {subject}

Body:
{body}
""".strip()
    
    return email_text

In [7]:
def traige_email(email_text: str) -> str:
    prompt = f"""
You are an expert AI Executive Assistant.
Your goal is to triage incoming emails into specific category

### CATEGORY DEFINITIONS:
1. **IGNORE**: Automated newsletters, marketing, receipts, system logs, or spam.
2. **NOTIFY**: informational emails where NO reply is expected (e.g., shipping updates, "FYI" memos, broad company announcements).
3. **RESPOND**: Emails that require a reply or action. This INCLUDES:
   - Invitations (Parties, Weddings, Meetings) that need an RSVP.**
   - Direct questions asked to Sanjay.
   - Scheduling requests.
   - Personal messages requiring acknowledgement.


Email:
{email_text}

Only return the category.
"""
    return llm_call(prompt)

In [41]:
# Calendar Tools
import datetime

def ensure_timezone(iso_time):
    """Helper to ensure time string ends with Z if no offset exists."""
    if not iso_time.endswith('Z') and '+' not in iso_time[-6:]:
        return iso_time + 'Z'
    return iso_time

def check_availability(service, start_time_iso):
    """
    Checks availability and returns the conflicting event object if busy.
    """
    # 1. Fix Timezone (Prevents 400 Error)
    start_time_iso = ensure_timezone(start_time_iso)

    # 2. Calculate End Time
    start = datetime.datetime.fromisoformat(start_time_iso.replace('Z', '+00:00'))
    end = start + datetime.timedelta(hours=1)
    end_time_iso = end.isoformat()
    
    # Ensure end time also has Z if needed
    end_time_iso = ensure_timezone(end_time_iso)

    try:
        events_result = service.events().list(
            calendarId='primary',
            timeMin=start_time_iso,
            timeMax=end_time_iso,
            singleEvents=True
        ).execute()

        events = events_result.get('items', [])
        
        # 3. Return Dictionary (Required for 'Replace' logic)
        if events:
            return {"status": "BUSY", "event": events[0]}
        
        return {"status": "FREE", "event": None}
        
    except Exception as e:
        print(f"Calendar API Check Error: {e}")
        return {"status": "ERROR", "event": None}

def add_event(service, summary, start_time_iso):
    """Adds an event to the primary calendar."""
    # 1. Fix Timezone
    start_time_iso = ensure_timezone(start_time_iso)
    
    start = datetime.datetime.fromisoformat(start_time_iso.replace('Z', '+00:00'))
    end = start + datetime.timedelta(hours=1)
    end_time_iso = end.isoformat()
    
    # Ensure end time also has Z
    end_time_iso = ensure_timezone(end_time_iso)

    event_body = {
        'summary': summary,
        'start': {'dateTime': start_time_iso},
        'end': {'dateTime': end_time_iso}
    }
    
    try:
        event = service.events().insert(calendarId='primary', body=event_body).execute()
        return f"Success: Event created at {event.get('htmlLink')}"
    except Exception as e:
        return f"Error creating event: {e}"

def delete_event(service, event_id):
    """Deletes an event by ID."""
    try:
        # Fixed typo: .delet -> .delete
        service.events().delete(calendarId='primary', eventId=event_id).execute()
        print(f"🗑️ Deleted conflicting event (ID: {event_id})")
        return True
    except Exception as e:
        print(f"Error deleting event: {e}")
        return False

In [46]:
import base64
from email.mime.text import MIMEText
import json
import datetime
from datetime import datetime as dt

def create_draft_reply(gmail_service, calendar_service, sender: str, subject: str, body: str) -> str:

    # Extract Event Time ---
    now = dt.now()
    current_date_str = now.strftime("%A, %B %d, %Y") # e.g., "Monday, January 26, 2026"
    current_year = now.year

    time_prompt = f"""
    Context: Today is {current_date_str}.
    
    Task: Extract the event start time from the email below.
    - If a date (like "28 Jan") is mentioned without a year, assume the year is {current_year}.
    - Convert the time to ISO 8601 format (YYYY-MM-DDTHH:MM:SS).
    - If NO specific time is found, return exactly 'NONE'.
    
    Email Body:
    "{body}"
    
    Return ONLY the ISO string or 'NONE'. No markdown, no quotes.
    """
    event_time = llm_call(time_prompt).strip().replace('"', '').replace("'", "")
    
    # Check Availability
    calendar_context = "No specific time mentioned in email."
    conflict_event = None
    is_free = True
    event_already_scheduled = False # Flag to prevent double-booking on 'replace'

    if event_time != "NONE" and "T" in event_time:
        print(f"\n📅 Detected Event Time: {event_time}")
        availability = check_availability(calendar_service, event_time)
        
        if availability["status"] == "BUSY":
            is_free = False
            conflict_event = availability["event"]
            calendar_context = f"WARNING: You are BUSY at this time. Conflicting Event: '{conflict_event['summary']}' (ID: {conflict_event['id']})."
        else:
            calendar_context = "You are FREE at this time."

    current_feedback = ""
    
    while True:
        prompt = f"""
        You are a professional Email Assistant for Sanjay Sanapala.
        Your goal is to write a professional email reply.

        ### INPUT DATA:
        - Sender: {sender}
        - Original Subject: {subject}
        - Original Body: {body}

        ### CALENDAR CONTEXT:
        {calendar_context}
        
        ### USER FEEDBACK / ADJUSTMENTS:
        {current_feedback if current_feedback else "None (Draft the initial reply)"}

        ### GUIDELINES:
        1. **Tone:** Professional, direct, and polite.
        2. **Structure:** Greeting -> Main Point -> Call to Action -> Sign off.
        3. **Constraint:** Return strictly valid JSON.

        ### OUTPUT FORMAT (JSON ONLY):
        {{
            "To": "{sender}",
            "Subject": "Re: {subject} (or a better subject)",
            "Body": "The full email body text here..."
        }}
        """

        print("\n--- Generating Draft... ---")
        
        try:
            response_json = llm_call(prompt)
            reply_data = json.loads(response_json)
        except json.JSONDecodeError:
            print("Error: LLM did not return valid JSON. Retrying...")
            continue

        print(f"\nDRAFT PREVIEW:\nTo: {reply_data['To']}\nSubject: {reply_data['Subject']}\nBody:\n{reply_data['Body']}\n")
        
        user_choice = input("Action (yes / no / replace / [type feedback]): ").strip().lower()

        if user_choice in ["yes", "y"]:
            print(f"📅 Adding event: 'Meeting with {sender}'...")
            add_event(calendar_service, f"Meeting with {sender}", event_time)
            
            print("Success: Draft Created & Calendar Updated." ) 
        # --- 2. HANDLE REPLACEMENT (REPLACE) ---
        elif "replace" in user_choice:
            # Check if there is actually a conflict to replace
            if conflict_event:
                print(f"🔄 Replacing conflicting event: '{conflict_event['summary']}'...")
                
                delete_event(calendar_service, conflict_event['id'])
                add_event(calendar_service, f"Meeting with {sender}", event_time)
                
                event_already_scheduled = True 
                is_free = True 
                conflict_event = None
                
                calendar_context = "Availability: FREE (User manually cleared the previous conflict). You MUST ACCEPT the invitation now."
                current_feedback += " I have cleared the calendar conflict. Rewrite the email to ACCEPT the invitation."
                
                print("Event swapped. Regenerating draft with acceptance...")
            
            else:
                print("⚠️ No conflicting event found to replace. (Calendar is already free or no time detected).")

        # --- 3. HANDLE CANCELLATION (NO) ---
        elif user_choice in ["no", "n"]:
            print("Operation cancelled.")
            return "Cancelled."

        # --- 4. HANDLE FEEDBACK (EVERYTHING ELSE) ---
        else:
            print("Refining draft based on feedback...")
            current_feedback += f"\n- User requested change: {user_choice}"

In [45]:
if __name__ == "__main__":
    messages = fetch_recent_emails(service, 1)

    for msg in messages:
        sender, subject, body = extract_email_parts(msg)

        mail_text = email_for_llm(sender, subject, body)

        decision = traige_email(mail_text).strip()
        decision = decision.replace('"', '').replace("'", "").strip()
        
        print(f"Decision: {decision}")

        if decision == "IGNORE":
            print("Mail Ignored")
            
        elif decision == "NOTIFY":
            print(f"Notification: {subject}")
            
        elif decision == "RESPOND":
            create_draft_reply(service, calendar_service, sender, subject, body)

Decision: RESPOND

📅 Detected Event Time: 2026-01-28T15:00:00

--- Generating Draft... ---

DRAFT PREVIEW:
To: L Air <lairsup090@gmail.com>
Subject: Re: Family meeting
Body:
Dear Lair,

Thank you for the invitation to the family meeting on January 28th from 3:00 PM to 5:00 PM. I appreciate you letting me know.

Unfortunately, I have a prior commitment during that time. I have a meeting scheduled with Bluey Face that I cannot reschedule.

Could there be any flexibility with the family meeting time? If not, I will do my best to join for a portion or arrange another time to see Grandpa.

Please let me know if there are any alternative arrangements.

Best regards,
Sanjay Sanapala

🔄 Replacing conflicting event: 'Meeting with Bluey Face <blueyface043@gmail.com>'...
🗑️ Deleted conflicting event (ID: m0pp2j31r228jti823miugdld8)
Event swapped. Regenerating draft with acceptance...

--- Generating Draft... ---

DRAFT PREVIEW:
To: L Air <lairsup090@gmail.com>
Subject: Re: Family meeting - Accept

In [15]:
def search_gmail(service, query, max_results=5):
    """Search Gmail using the 'q' parameter."""
    try:
        results = service.users().messages().list(
            userId="me",
            q=query,
            maxResults = max_results
        ).execute()

        messages = results.get("messages", [])

        if "resultSizeEstimate" in results and max_results == 1 and not messages:
            return [], results.get("resultSizeEstimate", 0)
        
        full_emails = []
        for msg in messages:
            message = service.users().messages().get(
                userId="me",
                id=msg["id"],
                format='full'
            ).execute()
            full_emails.append(message)

        return full_emails, len(messages)
    except Exception as e:
        print(f"Error: {e}")
        return [], 0

In [16]:
import json
from datetime import date

def query_gmail_assistant(user_question):
    today = date.today().strftime("%Y/%m/%d")

    prompt = f"""
    You are an expert Translator for the Gmail API.
    Your goal is to convert a user's natural language question into a Gmail search query (parameter 'q').

    ### CONTEXT:
    - Today's Date: {today}
    - User: Sanjay Sanapala

    ### GMAIL QUERY SYNTAX CHEATSHEET:
    - Unread emails: "is:unread"
    - From specific person: "from:name@example.com" or "from:Name"
    - Subject contains: "subject:(meeting)"
    - Date range: "after:2024/01/01 before:2024/01/31"
    - Labels: "label:Work", "label:Updates"
    - Categories: "category:primary", "category:social", "category:promotions"
    - Exact phrase: "\"exact phrase\""

    ### EXAMPLES:
    User: "How many unread emails do I have?"
    Output: {{"query": "is:unread", "intention": "count"}}

    User: "Show me the latest email from Sanjay."
    Output: {{"query": "from:Sanjay", "intention": "list"}}

    User: "Any emails about the internship in the last 2 days?"
    Output: {{"query": "internship after:{today}", "intention": "list"}}

    ### INPUT:
    User: "{user_question}"

    ### OUTPUT (JSON ONLY):
    Return valid JSON with keys: "query" (the gmail search string) and "intention" (either "count" or "list").
    """

    response = llm_call(prompt)

    try:
        return json.loads(response)
    except json.JSONDecodeError:
        return {"query": "", "intention": "error"}
    
def ask_gmail(service, query):
    print(f"User asking: '{query}'")

    res = query_gmail_assistant(query)
    gmail_query = res.get("query")
    intention = res.get("intention")

    print(f"Generated Query: '{gmail_query}' (Intention: {intention})")

    if not gmail_query:
        return "Sorry, I couldn't understand how to search for that"
    
    emails, count = search_gmail(service, gmail_query, max_results=5)

    if intention == "count":
        return f"I found approximately {count} emails matching your request"
    elif intention == "list":
        if not emails:
            return "No emails found matching that criteria."
        
        final_prompt = f"User asked: '{query}'.\nHere are the email details found:\n"

        for email in emails:
            sender, subject, body = extract_email_parts(email)
        
        final_prompt += "\n Answer the user's question naturally based on these emails."
        return llm_call(final_prompt)

In [26]:
emails, _ = search_gmail(service, "label:Starred")
for mail in emails:
    sender, subject, body = extract_email_parts(mail)
    print(email_for_llm(sender, subject, body))
    print("=" * 80)

Sender: Bluey Face <blueyface043@gmail.com>
Subject: Invitation to the party

Body:
Dear sanjay,

I hope you are doing well.

I am writing to cordially invite you to a party on 28 January at 7:00pm.
The event will be held at my house.

It would be wonderful to have you there to celebrate with us. Please let me
know if you will be able to attend.

I look forward to seeing you there.

Best regards,

Bluey
Sender: Bluey Face <blueyface043@gmail.com>
Subject: 

Body:
 datapot.vn-Practical-Statistics-for-Data-Scientists.pdf
<https://drive.google.com/file/d/1yrUoX8yfMyivefM2QUlu5hqcD3WNMh8g/view?usp=drive_web>
 Hands on Machine Learning with Scikit Learn and TensorFlow.pdf
<https://drive.google.com/file/d/1O9ZalR8QQrgMnStcx7OidlSuBzZwNeLq/view?usp=drive_web>

Machine.Learning.with.PyTorch.and.Scikit-Learn.Sebastian.Raschka.Packt.9781801819312.EBooksWorld....
<https://drive.google.com/file/d/1TWjiRad3C0hwTelx8XDLpjCA7rMxl8Dv/view?usp=drive_web>
 mml-book.pdf
<https://drive.google.com/file/d/1

In [27]:
response = ask_gmail(service, "How many unread emails are there")
print(response)

User asking: 'How many unread emails are there'
Generated Query: 'is:unread' (Intention: count)
I found approximately 5 emails matching your request
